In [1]:
from typing import List
from Dataset.CustomDataset import AgeGroupAndAgeDataset, StandardDataset
from Dataset.CustomDataLoaders import CustomDataLoader
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from Utils import AAR, CSVUtils, AgeConversion
from Utils.Validator import Validator

c:\Users\Gio\.conda\envs\AV_project\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Caricamento del dataframe
df = CSVUtils.get_df_from_csv("./training_caip_contest.csv", "./training_caip_contest/")

#Suddivisione del dataframe in 3 age groups
_, d = CSVUtils.get_df_with_age_subdivision(df, 3)

In [3]:
d

{1: 0,
 2: 0,
 3: 0,
 4: 0,
 5: 0,
 6: 0,
 7: 0,
 8: 0,
 9: 0,
 10: 0,
 11: 0,
 12: 0,
 13: 0,
 14: 0,
 15: 0,
 16: 0,
 17: 0,
 18: 0,
 19: 0,
 20: 0,
 21: 0,
 22: 0,
 23: 0,
 24: 0,
 25: 0,
 26: 0,
 27: 0,
 28: 0,
 29: 0,
 30: 0,
 31: 0,
 32: 1,
 33: 1,
 34: 1,
 35: 1,
 36: 1,
 37: 1,
 38: 1,
 39: 1,
 40: 1,
 41: 1,
 42: 1,
 43: 1,
 44: 1,
 45: 2,
 46: 2,
 47: 2,
 48: 2,
 49: 2,
 50: 2,
 51: 2,
 52: 2,
 53: 2,
 54: 2,
 55: 2,
 56: 2,
 57: 2,
 58: 2,
 59: 2,
 60: 2,
 61: 2,
 62: 2,
 63: 2,
 64: 2,
 65: 2,
 66: 2,
 67: 2,
 68: 2,
 69: 2,
 70: 2,
 71: 2,
 72: 2,
 73: 2,
 74: 2,
 75: 2,
 76: 2,
 77: 2,
 78: 2,
 79: 2,
 80: 2,
 81: 2}

In [3]:
#Splitting tra Train e Validation set
df_train, df_val = train_test_split(df, test_size=0.25, random_state=42)
#Aggiornamento degli indici per pandas
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)

from PIL import Image
import numpy as np
def transform_image(image: Image):
    #Transpose necessaria per adattare il formato di input a pyTorch 
    return (np.array(image.resize((224, 224))) / 255.0).transpose(2, 0, 1).astype(np.float32)

#Implementazione di un Dataset utilizzando "CustomDataset" per l'architettura con Film
cd_train = AgeGroupAndAgeDataset(df_train, path_col="path", label_col="age", label_function="CAE", 
                                 label_map=d, label_map_n_classes=3, transform_func=transform_image)

#Implementazione di un Dataset che adatta le label all'utilizzo che vogliamo farne (in questo caso CAE = Cathegorical)
cd_val = StandardDataset(df_val, path_col="path", label_col="age", label_function="CAE", transform_func=transform_image)

#Dato che lo split potrebbe non prendere sample di determinate classi facciamo il set del numero di classi
cd_val.set_n_classes(81)

#Loader che, conoscendo la grandezza del dataset, farà lo shuffle dei campioni e crea i batch
dm_train, dm_val = CustomDataLoader(cd_train), CustomDataLoader(cd_val)
#Utilizziamo il dataloader che crea batch bilanciati implementato in CustomDataLoaders
# In generale non usare questo ma l'unbalanced dato che non è usato in nessun doc
dl_train = dm_train.get_balanced_dataloader(class_ranges=[(0, 11), (11, 21), (21, 31), (31, 41), (41, 51), (51, 61), (61, 71), (71, 91)], samples_per_class=4)

In [4]:
from SE import se_module, se_resnet
import torch
from torch import optim
import torch.nn.functional as F
from torch import nn


class SEResNet50FiLMedGroups(nn.Module):
    def __init__(self, n_classes: int, backbone) -> None:
        super().__init__()

        self.model = backbone
        self.model.to("cuda")
        self.model_film_layers: List[nn.Module] = self._get_film_layers()
        self.set_grads()
        self.fc0 = nn.Linear(2048, 1024).to("cuda")
        self.model.fc = self.fc0
        self.act0 = nn.ReLU().to("cuda")
        self.fc1 = nn.Linear(1024, 1024).to("cuda")
        self.act1 = nn.ReLU().to("cuda")
        self.fc2 = nn.Linear(1024, n_classes).to("cuda")

    def _get_film_layers(self) -> List:
        model_layers = [self.model.layer1, self.model.layer2, self.model.layer3, self.model.layer4]
        model_film_layers = []
        for layer in model_layers:
            for layer_module in layer.modules():
                if not layer_module._get_name() == "SEBottleneck_FiLM":
                    continue
                model_film_layers += [x for x in layer_module.modules() if x._get_name() == "FiLMLayer"]
        return model_film_layers

    def set_grads(self) -> None:
        self.model.train()
        self.model.requires_grad_(True)
        for x in self.model_film_layers:
            x.requires_grad_(True)

    def set_knowledge(self, knowledge: torch.Tensor) -> None:
        for x in self.model_film_layers:
            x.set_knowledge(knowledge)

    def forward(self, x, knowledge):
        self.set_knowledge(knowledge)
        out = self.model(x)
        out = self.act0(out)
        out = self.fc1(out)
        out = self.act1(out)
        out = self.fc2(out)
        return out

bb = se_resnet.se_resnet50_filmed(1000, dim_knowledge=3, pretrained=True)
model_age_group = SEResNet50FiLMedGroups(3, bb)
model_age = SEResNet50FiLMedGroups(81, bb)
opt = optim.Adam(set([*model_age_group.parameters(), *model_age.parameters()]), lr=1e-4)
criterion = nn.CrossEntropyLoss()
# AAR non funziona nel caso di unbalanced loader
aar = AAR.AAR(4)

In [ ]:
#Validator che si occuperà di tenere in considerazione le metriche da massimizzare per il contest
validator = Validator(cd_val, AgeConversion.ArgMaxAge, 128)

In [ ]:
best_val_aar = -1

epochs = 10

knowledge_age_group = torch.tensor([[0.33, 0.33, 0.33]]*32, requires_grad=False).float().to("cuda")

for e in range(epochs):
    with tqdm(dl_train, unit=" batch") as tepoch:
        for batch in tepoch:
            opt.zero_grad()
            x, y = batch
            x = x.to("cuda")
            y_age_group = y[0].to("cuda")
            y_age = y[1].to("cuda")
            
            knowledge = model_age_group(x, knowledge_age_group)

            knowledge = F.softmax(knowledge, dim=-1)

            out = model_age(x, knowledge)

            loss_age_group: torch.Tensor = criterion(knowledge, y_age_group)
            loss: torch.Tensor = criterion(out, y_age)
            loss_age_group.backward(retain_graph=True)
            loss.backward()
            opt.step()

            # AAR non funziona nel caso di unbalanced loader
            tepoch.set_postfix(loss_age_group=loss_age_group.detach().cpu().numpy(), loss=loss.detach().cpu().numpy(),
                               AAR=aar(AgeConversion.ArgMaxAge(y_age.cpu()).float(), AgeConversion.ArgMaxAge(out.cpu()).float())) 

    def forward_function(x):
        knowledge_age_group = torch.tensor([[0.33, 0.33, 0.33]]*len(x), requires_grad=False).float().to("cuda")
        knowledge = model_age_group(x, knowledge_age_group)
        knowledge = F.softmax(knowledge, dim=-1)
        out = model_age(x, knowledge)
        out = F.softmax(out, dim=-1)

    val_aar, val_aar_old = validator.validate(forward_function)

    if val_aar > best_val_aar:
        best_val_aar = val_aar
        torch.save(model_age_group.state_dict(), "./model_age_group.pt")
        torch.save(model_age.state_dict(), "./model_age.pt")
        print("Saved model")

In [ ]:
model_age.load_state_dict(torch.load("./with_loss/model_age.pt"))
model_age_group.load_state_dict(torch.load("./with_loss/model_age_group.pt"))